In [1]:
import optuna
from backend.genetic_algorithm import *
from math import floor

/Users/stefannosek/Documents/GitHub/class_scheduling/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rooms = [
    Room("B9-F1", 60, "normal"),
    Room("B9-F2", 60, "normal"),
    Room("B9-F3", 60, "normal"),
    Room("B9-F4", 60, "normal"),
    Room("B9-F5", 60, "normal"),
    Room("B9-F6", 30, "normal"),
    Room("B9-F7", 60, "normal"),
    Room("B9-F8", 60, "normal"),
    Room("B9-F9", 60, "normal"),
    Room("B9-F10", 150, "normal"),
    Room("B9-51", 30, "laboratories"),
    Room("B9-52", 60, "normal"),
    Room("B9-53", 30, "laboratories"),
    Room("B14-1.11", 60, "normal"),
    Room("B14-1.12", 30, "laboratories"),
    Room("B14-2.1", 30, "laboratories"),
    Room("B14-2.2", 30, "laboratories"),
    Room("B14-2.3", 30, "laboratories"),
]

program = Data("program.xlsx")

professors = program.create_professors()

courses = program.create_courses(professors)

MS_sem_1_gr_1 = Students(0, 1, 1, "matematyka stosowana")
MS_sem_1_gr_2 = Students(1, 1, 2, "matematyka stosowana")
MS_sem_2_gr_1 = Students(2, 2, 1, "matematyka stosowana")
MS_sem_2_gr_2 = Students(3, 2, 2, "matematyka stosowana")
MS_sem_3_gr_1 = Students(4, 3, 1, "matematyka stosowana")
MS_sem_3_gr_2 = Students(5, 3, 2, "matematyka stosowana")
MS_sem_4_gr_1 = Students(6, 4, 1, "matematyka stosowana")
MS_sem_4_gr_2 = Students(7, 4, 2, "matematyka stosowana")
MS_sem_5_gr_1 = Students(8, 5, 1, "matematyka stosowana")
MS_sem_6_gr_1 = Students(9, 6, 1, "matematyka stosowana")

students = [
    MS_sem_1_gr_1, MS_sem_1_gr_2, MS_sem_2_gr_1, MS_sem_2_gr_2,
    MS_sem_3_gr_1, MS_sem_3_gr_2, MS_sem_4_gr_1, MS_sem_4_gr_2,
    MS_sem_5_gr_1, MS_sem_6_gr_1, 
]

In [3]:
def objective(trial):
    
    rand.seed(240)
    
    mutation_probability = trial.suggest_float("mutation_probability", 0.1, 1, step = 0.05)
    elitism_percentage = trial.suggest_float("elitism_percentage", 0.05, 0.2, step = 0.05)
    mating_percentage = trial.suggest_float("mating_percentage", 0.2, 1, step = 0.1)
    number_of_class_swaps = trial.suggest_int("number_of_class_swaps", 1, 15, step = 1)
    number_of_room_swaps = trial.suggest_int("number_of_room_swaps", 1, 15, step = 1)

    pop = Population(
        250,
        professors,
        courses,
        rooms,
        students,
        mutation_probability,
        elitism_percentage,
        mating_percentage,
        number_of_class_swaps,
        number_of_room_swaps,
    )

    pop.population = pop.sort_by_fitness_and_soft_fitness(pop.population)

    pop.execute(100)
    
    return 5 * pop.population[0].fitness + pop.population[0].soft_fitness

In [4]:
study = optuna.create_study(direction = "maximize")

[I 2024-01-26 00:23:08,745] A new study created in memory with name: no-name-bc8a589e-a250-44d6-a37b-97ad66b9a4ab


In [5]:
study.optimize(objective, n_trials = 200)

[I 2024-01-26 00:24:42,415] Trial 0 finished with value: 5.113924050632911 and parameters: {'mutation_probability': 0.75, 'elitism_percentage': 0.05, 'mating_percentage': 0.9000000000000001, 'number_of_class_swaps': 4, 'number_of_room_swaps': 5}. Best is trial 0 with value: 5.113924050632911.
[I 2024-01-26 00:25:43,142] Trial 1 finished with value: 5.115384615384615 and parameters: {'mutation_probability': 0.75, 'elitism_percentage': 0.05, 'mating_percentage': 0.4, 'number_of_class_swaps': 12, 'number_of_room_swaps': 5}. Best is trial 1 with value: 5.115384615384615.
[I 2024-01-26 00:26:50,155] Trial 2 finished with value: 5.125 and parameters: {'mutation_probability': 0.75, 'elitism_percentage': 0.05, 'mating_percentage': 0.5, 'number_of_class_swaps': 11, 'number_of_room_swaps': 7}. Best is trial 2 with value: 5.125.
[I 2024-01-26 00:28:16,574] Trial 3 finished with value: 5.211538461538462 and parameters: {'mutation_probability': 0.35, 'elitism_percentage': 0.2, 'mating_percentage': 

In [6]:
best_trial = study.best_trial

In [7]:
best_params = best_trial.params

In [8]:
best_params

{'mutation_probability': 0.75,
 'elitism_percentage': 0.2,
 'mating_percentage': 0.30000000000000004,
 'number_of_class_swaps': 3,
 'number_of_room_swaps': 1}